<a href="https://colab.research.google.com/github/ddgarciad/se-ales/blob/main/Taller_Pre_Parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yt-dlp pydub numpy matplotlib scipy simpleaudio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.1/172.1 kB 613.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.7 MB/s eta 0:00:00
  Created wheel for simpleaudio: filename=simpleaudio-1.0.4-cp310-cp310-linux_x86_64.whl size=2054405 sha256=a58f0840a55ae110da89c088034c54f590d55acd75986b73d338775ec356282e
  Stored in directory: /root/.cache/pip/wheels/10/70/ed/8c41675109565c2c65c3ac40d20859fc9d3f93a8efbb11d1c7
Successfully built simpleaudio


In [2]:
pip install simpleaudio


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import soundfile as sf
from scipy.fft import fft
from pydub import AudioSegment

# Intentar importar yt_dlp y manejar la falta de instalación
try:
    import yt_dlp
except ImportError:
    print("El módulo 'yt_dlp' no está instalado. Por favor, instálalo usando 'pip install yt-dlp'.")
    exit()

# Descargar el audio de YouTube y convertir a WAV
def descargar_audio_youtube(url, output_filename='audio.wav'):
    ydl_opts = {
        'format': 'bestaudio[ext=m4a]',
        'outtmpl': 'audio.m4a'
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

    # Convertir a WAV
    audio = AudioSegment.from_file("audio.m4a", format="m4a")
    audio.export(output_filename, format="wav")
    os.remove("audio.m4a")  # Limpiar el archivo m4a temporal

# Cargar el archivo de audio
def cargar_audio(filename):
    try:
        data, sample_rate = sf.read(filename)
        return sample_rate, data
    except FileNotFoundError:
        print(f"No se encontró el archivo {filename}. Asegúrate de que el archivo exista y esté en la ubicación correcta.")
        return None, None

# Convertir audio estéreo a mono
def convertir_a_mono(data):
    if len(data.shape) > 1:  # Si es estéreo
        return np.mean(data, axis=1)
    return data

# Graficar la señal en el tiempo
def graficar_senal_tiempo(t, signal, title, xlabel, ylabel):
    plt.figure(figsize=(10, 4))
    plt.plot(t, signal)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid()
    plt.show()

# Graficar la transformada de Fourier
def graficar_fft(t, signal, title):
    N = len(signal)
    T = t[1] - t[0]
    yf = fft(signal)
    xf = np.fft.fftfreq(N, T)[:N // 2]

    plt.figure(figsize=(10, 4))
    plt.plot(xf, np.abs(yf[:N // 2]))
    plt.title(title)
    plt.xlabel('Frecuencia [Hz]')
    plt.ylabel('Magnitud')
    plt.grid()
    plt.show()

# Guardar la señal en un archivo WAV
def guardar_audio(data, sample_rate, filename):
    sf.write(filename, data, sample_rate)
    print(f"Audio guardado como '{filename}'")

# Función principal
def main():
    # Solicitar el enlace de YouTube
    url = input("Introduce el enlace del video de YouTube: ")
    output_wav = 'audio.wav'

    print("Descargando audio de YouTube...")
    descargar_audio_youtube(url, output_wav)

    # Cargar el archivo de audio convertido
    sample_rate, data = cargar_audio(output_wav)
    if data is None:
        return  # Salir si el archivo no se encuentra

    # Convertir a mono si es estéreo
    data_segment = convertir_a_mono(data)

    # Leer un fragmento de 5 segundos del archivo de audio
    ti = 20  # tiempo de inicio en segundos
    tf = 25  # tiempo final en segundos
    data_segment = data_segment[int(ti * sample_rate):int(tf * sample_rate)]

    # Tiempo de la señal
    t = np.arange(len(data_segment)) / sample_rate

    # Mostrar la señal de audio original
    print("Graficando la señal de audio original...")
    graficar_senal_tiempo(t, data_segment, 'Señal de Audio en el Tiempo', 'Tiempo [s]', 'Amplitud')

    # Guardar el audio original
    guardar_audio(data_segment, sample_rate, "original_segment.wav")

    # Transformada de Fourier de la señal de audio
    print("Graficando el espectro de la señal de audio...")
    graficar_fft(t, data_segment, 'Espectro de la Señal de Audio')

    # Parámetros de la portadora
    Fc = float(input("Introduce la frecuencia de la portadora (Hz): "))
    Im = float(input("Introduce el índice de modulación (por ejemplo, 0.5): "))
    Ac = np.max(np.abs(data_segment)) / Im

    # Generar la señal portadora
    c = Ac * np.sin(2 * np.pi * Fc * t)

    # Señal modulada
    y = (1 + data_segment / Ac) * c

    # Mostrar la señal portadora
    print("Graficando la señal portadora...")
    graficar_senal_tiempo(t, c, 'Señal Portadora en el Tiempo', 'Tiempo [s]', 'Amplitud')

    # Guardar la señal portadora
    guardar_audio(c, sample_rate, "carrier_signal.wav")

    # Transformada de Fourier de la señal portadora
    print("Graficando el espectro de la señal portadora...")
    graficar_fft(t, c, 'Espectro de la Señal Portadora')

    # Mostrar la señal modulada
    print("Graficando la señal modulada AM...")
    graficar_senal_tiempo(t, y, 'Señal Modulada AM en el Tiempo', 'Tiempo [s]', 'Amplitud')

    # Guardar la señal modulada
    guardar_audio(y, sample_rate, "modulated_signal.wav")

    # Transformada de Fourier de la señal modulada
    print("Graficando el espectro de la señal modulada AM...")
    graficar_fft(t, y, 'Espectro de la Señal Modulada')

    # Eliminar el archivo temporal
    os.remove(output_wav)

if __name__ == "__main__":
    main()
